In [ ]:
pip install tensorflow-determinism

In [ ]:
pip install tsfeatures

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 25.9 MB/s eta 0:00:00
  Created wheel for supersmoother: filename=supersmoother-0.4-py3-none-any.whl size=16146 sha256=5bdacc5ea9495a05620c1df9162e60ebb5941c28635e662c9262fe4b31be8544
  Stored in directory: /root/.cache/pip/wheels/ce/54/3b/3a6a20584d667a3a898b09ead38658bc5b8b3f68cb2875fb6d
Successfully built supersmoother


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, LSTM, Dense, Activation, Dropout, Embedding, Concatenate, Add, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
# from tensorflow.keras import optimizers
import time
import tensorflow as tf

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf
# from pmdarima.arima import auto_arima

from tqdm import tqdm
import random
import os
import copy

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from tsfeatures import tsfeatures
from tsfeatures import acf_features, pacf_features, entropy, lumpiness, stl_features, arch_stat, nonlinearity, unitroot_kpss, unitroot_pp, holt_parameters, hw_parameters

In [ ]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

In [ ]:
def split_windows(x_lst, num_series, input_len, output_len):
  num_window_per_series = np.zeros(num_series, dtype=int)
  # Calculate the number of windows per series
  for i in range(num_series):
      num_window_per_series[i] = x_lst[i].shape[0] - input_len
  # Calculate the total number of windows
  num_window_total = np.sum(num_window_per_series)

  # Initialize the data frame
  x_data_frame = np.zeros((num_window_total, input_len + output_len))

  # Populate the data_frame
  win = 0
  for k in range(num_series):
      for win_k in range(num_window_per_series[k]):
          x_data_frame[win, :] = x_lst[k].loc[
              win_k:(win_k + input_len + output_len-1), 'data'
          ] # 0:30 include the last value
          win += 1

  # Initialize mean and standard deviation vectors
  mean_vec = np.zeros(num_window_total)
  std_vec = np.zeros(num_window_total)
  # Calculate mean and standard deviation for each window
  for win in range(num_window_total):
      mean_vec[win] = np.mean(x_data_frame[win, :input_len]) # :30, exclude the last one
      std_vec[win] = np.std(x_data_frame[win, :input_len])
  # Normalize the data frame
  norm_x_data_frame = np.zeros_like(x_data_frame)
  for win in range(num_window_total):
      norm_x_data_frame[win, :] = (
          x_data_frame[win, :] - mean_vec[win]
      ) / std_vec[win]

  return([x_data_frame, norm_x_data_frame, num_window_per_series, mean_vec, std_vec])

In [ ]:
def pred_and_results(x, model, x_data_frame, output_len, mean_vec_x, std_vec_x):
  x_predict = model.predict(x).reshape(-1)
  x_true = x_data_frame[:, -output_len]
  # Initialize an array for the original predictions
  orig_pred_x = np.zeros(len(x_predict))

  # Transform predictions back to the original scale
  for i in range(len(x_predict)):
      orig_pred_x[i] = x_predict[i] * std_vec_x[i] + mean_vec_x[i]

  return([orig_pred_x, x_true])

In [ ]:
def split_to_series(a, num_per_series, num_series):
  a_lst = []
  # Loop through each series
  for i in range(num_series):
      a_lst.append(a[:num_per_series[i]]) # exclude the last one
      a = a[num_per_series[i]:]  # Remove used elements from a
  return(a_lst)

In [ ]:
def metrics(num_series, true_demand_lst, pred_test_demand_lst):
  # Initialize arrays to store metrics
  rmse_vec = np.zeros(num_series)
  mae_vec = np.zeros(num_series)
  smape_vec = np.zeros(num_series)

  # Compute metrics for each series
  for i in range(num_series):
      true_values = true_demand_lst[i]
      predicted_values = pred_test_demand_lst[i]

      # RMSE
      rmse_vec[i] = np.sqrt(np.mean((predicted_values - true_values) ** 2))

      # MAE
      mae_vec[i] = np.mean(np.abs(predicted_values - true_values))

      # sMAPE
      smape_vec[i] = 2 * np.mean(
          np.abs(predicted_values - true_values) /
          (np.abs(predicted_values) + np.abs(true_values))
      )

  # Aggregate results
  mean_rmse = np.mean(rmse_vec)
  median_rmse = np.median(rmse_vec)

  mean_mae = np.mean(mae_vec)
  median_mae = np.median(mae_vec)

  mean_smape = np.mean(smape_vec)
  median_smape = np.median(smape_vec)

  # # Print results
  # print("Mean RMSE:", mean_rmse)
  # print("Median RMSE:", median_rmse)

  # print("Mean MAE:", mean_mae)
  # print("Median MAE:", median_mae)

  # print("Mean sMAPE:", mean_smape)
  # print("Median sMAPE:", median_smape)

  # cum_rmse
  cum_rmse_lst = [
    np.mean([
        np.sqrt(
            np.mean((np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1]))**2)
        )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  cum_mae_lst = [
    np.mean([
        np.mean(
            np.abs(np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1]))
            )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  cum_smape_lst = [
    np.mean([
        2* np.mean(
            np.abs(np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1])) /
            (np.abs(np.array(true_demand_lst[i][:t+1])) + np.abs(np.array(pred_test_demand_lst[i][:t+1])))
            )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  mean_cum_rmse = np.mean(cum_rmse_lst)
  median_cum_rmse = np.median(cum_rmse_lst)
  mean_cum_mae = np.mean(cum_mae_lst)
  median_cum_mae = np.median(cum_mae_lst)
  mean_cum_smape = np.mean(cum_smape_lst)
  median_cum_smape = np.median(cum_smape_lst)

  # print("Mean Cum RMSE:", mean_cum_rmse)
  # print("Median Cum RMSE:", median_cum_rmse)

  return([mean_rmse, median_rmse, mean_mae, median_mae, mean_smape, median_smape,
          mean_cum_rmse, median_cum_rmse, mean_cum_mae, median_cum_mae, mean_cum_smape, median_cum_smape])

In [ ]:
def build_mlp_model(input_len, output_len):
  inputs = Input(shape=(input_len,))
  x = Dense(8, activation="tanh")(inputs)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
def build_lstm_model(input_len, output_len):
  inputs = Input(shape=(input_len,1))
  x = LSTM(units = 16, return_sequences = True)(inputs)
  x = Dropout(0.2)(x)
  x = LSTM(units = 8, return_sequences = False)(x)
  x = Dropout(0.2)(x)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
def build_cluster_model_V1(model, input_len, output_len):
  input_layer_model_A = model.input
  output_model_A = model.layers[-2].output # Exclude the last layer / output layer
  x = Dense(input_len, activation='tanh', name='new_dense_1')(output_model_A)
  x = Add()([x, Reshape((input_len,))(input_layer_model_A)])
  x = Dense(2, activation='tanh', name='new_dense_2')(x)
  output_clu = Dense(output_len, name='new_dense_3')(x)
  model_clu = Model(inputs=input_layer_model_A, outputs=output_clu)

  for i in range(len(model.layers) - 1):  # Exclude the output layer of the global model
      model_clu.layers[i].set_weights(model.layers[i].get_weights())
  # Freeze layers
  for layer in model_clu.layers[:(len(model.layers) - 1)]:
      layer.trainable = False

  # model_clu.summary()
  return(model_clu)

In [ ]:
def run_cluster_models(required_num_of_clusters, hetero_series_idx, cluster_labels,
                       xtrain_lst, xtest_lst, num_series,
                       input_len, output_len,
                       model, epochs_stage_2, batch_size_stage_2,
                       pred_lst, train_true_lst,
                       test_pred_lst, test_true_lst,
                       num_window_per_series_train, num_window_per_series_test,
                       random_seed):
  count_cluster, num_windosw_cluster, num_hetero_cluster = [], [], []
  test_pred_with_adj_lst = copy.deepcopy(test_pred_lst)

  for idx_cluster in range(required_num_of_clusters):
    cluster_i = hetero_series_idx[np.where(cluster_labels == idx_cluster)]
    count_cluster.append(len(cluster_i))

    cluster_lst = []
    for i in cluster_i:
      cluster_lst.append(xtrain_lst[i])
    cluster_test_lst = []
    for i in cluster_i:
      cluster_test_lst.append(xtest_lst[i])

    # prepare training data
    train_data_frame_clu, norm_train_data_frame_clu, num_window_per_series_train_clu, mean_vec_train_clu, std_vec_train_clu = \
    split_windows(cluster_lst, len(cluster_i), input_len, output_len)
    num_windosw_cluster.append(np.sum(num_window_per_series_train_clu))
    # [batch, timesteps, feature]
    train_X_clu = norm_train_data_frame_clu[:,:-output_len].reshape((-1,input_len,1))
    train_Y_clu = norm_train_data_frame_clu[:,-output_len].reshape((-1,output_len,1))

    # if np.sum(num_window_per_series_train_clu) <= 4000:
    #   model_clu = build_cluster_model(model, output_len, complex=False)
    # else:
    #   model_clu = build_cluster_model(model, output_len, complex=True)
    model_clu = build_cluster_model_V1(model, input_len, output_len)
    print(model_clu.summary())
    # Compile the model
    model_clu.compile(loss = 'mean_squared_error', optimizer = 'adam')
    # Fine-tune the model
    set_global_determinism(seed=random_seed)
    model_clu.fit(train_X_clu, train_Y_clu, epochs=epochs_stage_2, batch_size=batch_size_stage_2)

    ##### evaluation on training dataset for this cluster
    train_pred_clu, train_true_clu = pred_and_results(train_X_clu, model_clu, train_data_frame_clu, output_len,
                                                      mean_vec_train_clu, std_vec_train_clu)
    train_pred_lst_clu = [pred_lst[i] for i in cluster_i]
    train_true_lst_clu = [train_true_lst[i] for i in cluster_i]
    # train, without adj
    train_metrics_clu_without_adj = metrics(len(cluster_i), train_true_lst_clu, train_pred_lst_clu)

    train_pred_clu_lst = split_to_series(train_pred_clu, num_window_per_series_train[cluster_i], len(cluster_i))
    # train, with adj
    train_metrics_clu_with_adj = metrics(len(cluster_i), train_true_lst_clu, train_pred_clu_lst)

    ##### detect heteroscedastic series for this cluster
    diff_lst_with_adj = [np.array([train_true_lst_clu[i] - train_pred_clu_lst[i]]).reshape(-1) for i in range(len(train_true_lst_clu))]
    # Initialize p-value vector
    p_vec_clu = np.zeros(len(cluster_i))
    # Perform the Ljung-Box test for each series
    for i in range(len(cluster_i)):
        lb_test = acorr_ljungbox(diff_lst_with_adj[i], lags=int(np.log(len(diff_lst_with_adj[i]))), return_df=True)
        p_vec_clu[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
    # Count and get indices of series with p-values < 0.05
    hetero_series_idx_clu = np.where(p_vec_clu < 0.05)[0]
    print(f"Number of heteroscedastic series: {len(hetero_series_idx_clu)}")
    num_hetero_cluster.append(len(hetero_series_idx_clu))

    ##### evaluation on test dataset for this cluster
    test_pred_lst_clu = [test_pred_lst[i] for i in cluster_i]
    test_true_lst_clu = [test_true_lst[i] for i in cluster_i]
    # test, without adj
    test_metrics_clu_without_adj = metrics(len(cluster_i), test_true_lst_clu, test_pred_lst_clu)

    # test, with adj
    test_data_frame_clu, norm_test_data_frame_clu, num_window_per_series_test_clu, mean_vec_test_clu, std_vec_test_clu = \
    split_windows(cluster_test_lst, len(cluster_i), input_len, output_len)
    test_X_clu = norm_test_data_frame_clu[:,:-output_len].reshape((-1,input_len,1))
    test_pred_clu, _ = pred_and_results(test_X_clu, model_clu, test_data_frame_clu, output_len,
                                        mean_vec_test_clu, std_vec_test_clu)
    test_pred_clu_lst = split_to_series(test_pred_clu, num_window_per_series_test[cluster_i], len(cluster_i))
    test_metrics_clu_with_adj = metrics(len(cluster_i), test_true_lst_clu, test_pred_clu_lst)

    ##### substitute and update the predictions of this cluster
    for i in cluster_i:
      test_pred_with_adj_lst[i] = test_pred_clu_lst[np.where(cluster_i == i)[0][0]]

    df = pd.DataFrame({'train_metrics_clu_without_adj': train_metrics_clu_without_adj,
                      'train_metrics_clu_with_adj': train_metrics_clu_with_adj,
                      'test_metrics_clu_without_adj': test_metrics_clu_without_adj,
                      'test_metrics_clu_with_adj': test_metrics_clu_with_adj})
    print('cluster', idx_cluster, df)

  test_metrics_with_adj = metrics(num_series, test_true_lst, test_pred_with_adj_lst)
  print(count_cluster, num_windosw_cluster, num_hetero_cluster)
  return(test_metrics_with_adj)

In [ ]:
def stage_2(xtrain_lst, xtest_lst, num_series, input_len, output_len,
            model, epochs_stage_2, batch_size_stage_2,
            pred_lst, train_true_lst, test_pred_lst, test_true_lst,
            hetero_series_idx, diff_lst,
            seasonal_period, random_seed,
            num_window_per_series_train, num_window_per_series_test,
            required_num_of_clusters = 5):
  set_global_determinism(seed=random_seed)
  print('clustering into', required_num_of_clusters, 'clusters')
  hetero_df = pd.DataFrame({'unique_id': [], 'ds': [], 'y': []})
  for i in hetero_series_idx:
    hetero_df_i = {'unique_id': [int(i) for j in range(len(diff_lst[i]))],
                  'ds': list(xtrain_lst[i].loc[(input_len):, 'time']),
                  'y': list(diff_lst[i])}
    hetero_df = pd.concat([hetero_df, pd.DataFrame(hetero_df_i)], axis=0, ignore_index=True)
  selected_features = tsfeatures(hetero_df, freq=seasonal_period,
                                 features=[acf_features, pacf_features, entropy, lumpiness, stl_features, arch_stat,
                                           nonlinearity, unitroot_kpss, unitroot_pp, holt_parameters, hw_parameters])
  # Select specific features
  selected_features = selected_features[[
      'hw_alpha', 'hw_beta', 'hw_gamma', 'alpha', 'beta',
      'trend', 'spike', 'linearity', 'curvature', 'entropy',
      'x_acf1', 'lumpiness'
  ]]
  # Apply Min-Max normalization
  scaler = MinMaxScaler()
  normalized_features = scaler.fit_transform(selected_features)
  normalized_features = pd.DataFrame(normalized_features, columns=selected_features.columns)
  # Remove columns containing NaN values
  normalized_features = normalized_features.dropna(axis=1)
  # divide domains
  kmeans = KMeans(n_clusters=required_num_of_clusters, random_state=0).fit(normalized_features)
  cluster_labels = kmeans.labels_

  print('building models for each cluster')
  test_metrics_with_adj = run_cluster_models(required_num_of_clusters, hetero_series_idx, cluster_labels,
                                             xtrain_lst, xtest_lst, num_series, input_len, output_len,
                                             model, epochs_stage_2, batch_size_stage_2,
                                             pred_lst, train_true_lst,
                                             test_pred_lst, test_true_lst,
                                             num_window_per_series_train, num_window_per_series_test,
                                             random_seed)
  return(test_metrics_with_adj)

In [ ]:
def run_global_domain_global(data, info, input_len, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 16,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 5,
                             random_seed = 42):
  num_series = info.shape[0]
  print('num_series:', num_series)

  data_lst = []
  for i in range(num_series):
    data_lst.append(data[data['idx']==(i+1)])

  xtest_idx = np.array(info['test_idx'])
  xtest_lst, xtrain_lst = [], []
  for i in range(num_series):
    xtrain_lst.append(data_lst[i][data_lst[i]['time']<xtest_idx[i]].reset_index(drop=True))
    xtest_lst.append(data_lst[i][data_lst[i]['time']>=xtest_idx[i] - input_len].reset_index(drop=True))

  train_data_frame, norm_train_data_frame, num_window_per_series_train, mean_vec_train, std_vec_train = \
  split_windows(xtrain_lst, num_series, input_len, output_len)
  # Calculate the total number of windows
  num_window_total_train = np.sum(num_window_per_series_train)
  print('total number of windows:', num_window_total_train)

  # [batch, timesteps, feature]
  train_X = norm_train_data_frame[:,:-output_len].reshape((-1,input_len,1))
  train_Y = norm_train_data_frame[:,-output_len].reshape((-1,output_len,1))

  tf.keras.backend.clear_session()
  # Call the above function with seed value
  set_global_determinism(seed=random_seed)

  if load_model_name == None:
    print('start training')
    model = build_model(input_len, output_len)
    history = model.fit(train_X, train_Y, batch_size = batch_size_stage_1, epochs = epochs_stage_1,
                        verbose=1, shuffle=False)
  else:
    print('loading model')
    model = tf.keras.models.load_model(load_model_name)
    print('done')

  orig_pred_train, train_true = \
  pred_and_results(train_X, model, train_data_frame, output_len, mean_vec_train, std_vec_train)
  train_rmse_without_adj = np.sqrt(np.mean((orig_pred_train - train_true)**2))
  print('train_rmse_without_adj:', train_rmse_without_adj)

  # detect heteroscedastic series
  diff = train_true / orig_pred_train # multiplicative model
  diff_lst = split_to_series(diff, num_window_per_series_train, num_series)
  pred_lst = split_to_series(orig_pred_train, num_window_per_series_train, num_series)
  # Initialize p-value vector
  p_vec = np.zeros(num_series)
  # Perform the Ljung-Box test for each series
  for i in range(num_series):
      lb_test = acorr_ljungbox(diff_lst[i], lags=int(np.log(len(diff_lst[i]))), return_df=True)
      p_vec[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
  # Count and get indices of series with p-values < 0.05
  hetero_series_idx = np.where(p_vec < 0.05)[0]
  print(f"Number of heteroscedastic series: {len(hetero_series_idx)}")

  train_true_lst = split_to_series(train_data_frame[:, -output_len], num_window_per_series_train, num_series)
  # test for the entire dataset
  print('start testing')
  test_data_frame, norm_test_data_frame, num_window_per_series_test, mean_vec_test, std_vec_test = \
  split_windows(xtest_lst, num_series, input_len, output_len)
  test_X = norm_test_data_frame[:, :input_len].reshape((-1,input_len,1))
  test_pred, test_true = \
  pred_and_results(test_X, model, test_data_frame, output_len, mean_vec_test, std_vec_test)
  test_pred_lst = split_to_series(test_pred, num_window_per_series_test, num_series)
  test_true_lst = split_to_series(test_true, num_window_per_series_test, num_series)
  test_metrics_without_adj = metrics(num_series, test_true_lst, test_pred_lst)

  # adj model
  print('entering stage 2')
  test_metrics_with_adj = stage_2(xtrain_lst, xtest_lst, num_series, input_len, output_len,
                                  model, epochs_stage_2, batch_size_stage_2,
                                  pred_lst, train_true_lst, test_pred_lst, test_true_lst,
                                  hetero_series_idx, diff_lst,
                                  seasonal_period, random_seed,
                                  num_window_per_series_train, num_window_per_series_test,
                                  required_num_of_clusters = required_num_of_clusters)
  metrics_res = {'res_without_adj': test_metrics_without_adj,
               'res_with_adj': test_metrics_with_adj}
  metrics_res_df = pd.DataFrame(metrics_res)
  return(metrics_res_df)

In [ ]:
data = pd.read_csv('tourism_data.csv')
info = pd.read_csv('tourism_info.csv')

In [ ]:
# tourism, lstm
# multiple random seed commands
run_global_domain_global(data, info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = 'lstm_global_tourism.keras', build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 16,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 5,
                             random_seed = 42)

num_series: 366
total number of windows: 91712
loading model
done
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step
train_rmse_without_adj: 4757.5427954533525
Number of heteroscedastic series: 162
start testing
275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
entering stage 2
clustering into 5 clusters
building models for each cluster


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 24, 16)         │          1,152 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 24, 16)         │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 8)              │            800 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 8)              │              0 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 4)              │             36 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_1 (Dense)       │ (None, 24)             │            120 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 24)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 24)             │              0 │ new_dense_1[0][0],     │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_2 (Dense)       │ (None, 2)              │             50 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_3 (Dense)       │ (None, 1)              │              3 │ new_dense_2[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,161 (8.44 KB)

 Trainable params: 173 (692.00 B)

...

 Total params: 2,161 (8.44 KB)

 Trainable params: 173 (692.00 B)

 Non-trainable params: 1,988 (7.77 KB)

None
Epoch 1/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.4122
Epoch 2/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.9884
Epoch 3/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.8677
Epoch 4/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.8196
Epoch 5/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.7922
Epoch 6/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.7728
Epoch 7/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.7598
Epoch 8/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.7490
Epoch 9/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.7406
Epoch 10/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.7350
Epoch 11/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.7307
Epoch 12/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.7260
Epoch 13/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.7217
Epoch 14/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.7183
Epoch 15/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/s

,res_without_adj,res_with_adj
0,1979.054546,1891.012578
1,465.065799,475.875338
2,1508.333659,1436.738760
3,369.222458,371.372236
4,0.164957,0.166043
5,0.141929,0.140903
6,1963.280541,1936.887337
7,495.335867,495.619751
8,1540.164182,1527.789497
9,384.459229,395.449964


In [ ]:
# tourism mlp, final
# multiple random seed commands
def build_mlp_model(input_len, output_len):
  inputs = Input(shape=(input_len,))
  x = Dense(8, activation="tanh")(inputs)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

# run_main(tourism_data, tourism_info, input_len = 24, output_len = 1, seasonal_period = 12,
#          build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

run_global_domain_global(data, info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 16,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 5,
                             random_seed = 3407)

num_series: 366
total number of windows: 91712
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8)                   │             200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

...

 Total params: 409 (1.60 KB)

 Trainable params: 173 (692.00 B)

 Non-trainable params: 236 (944.00 B)

None
Epoch 1/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.6841
Epoch 2/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9004
Epoch 3/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7346
Epoch 4/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6704
Epoch 5/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6304
Epoch 6/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6006
Epoch 7/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5758
Epoch 8/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5540
Epoch 9/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5351
Epoch 10/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5191
Epoch 11/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5061
Epoch 12/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4961
Epoch 13/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4884
Epoch 14/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4826
Epoch 15/40
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

,res_without_adj,res_with_adj
0,1876.497368,1860.880417
1,487.126565,481.024495
2,1446.438951,1427.059544
3,380.088163,375.435982
4,0.166789,0.166846
5,0.139166,0.139051
6,1913.457906,1892.542746
7,511.050519,499.366837
8,1539.330574,1513.959565
9,393.885128,375.761485


In [ ]:
cif_data = pd.read_csv('cif2016_data.csv')
cif_info = pd.read_csv('cif2016_info.csv')

In [ ]:
# cif mlp
def build_mlp_model(input_len, output_len):
  inputs = Input(shape=(input_len,))
  x = Dense(8, activation="tanh")(inputs)
  # x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

run_global_domain_global(cif_data, cif_info, input_len = 12, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 2,
                             random_seed = 3407)

num_series: 72
total number of windows: 5470
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 12)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8)                   │             104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 113 (452.00 B)

 Trainable params: 113 (452.00 B)

...

 Total params: 241 (964.00 B)

 Trainable params: 137 (548.00 B)

 Non-trainable params: 104 (416.00 B)

None
Epoch 1/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.5381
Epoch 2/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.7991
Epoch 3/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.4620
Epoch 4/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.2513
Epoch 5/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.1109
Epoch 6/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.0122
Epoch 7/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.9357
Epoch 8/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.8694
Epoch 9/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.8099
Epoch 10/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7600
Epoch 11/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.7198
Epoch 12/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.6863
Epoch 13/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.6565
Epoch 14/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6283
Epoch 15/40
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

,res_without_adj,res_with_adj
0,352476.767014,331344.180616
1,86.941856,72.154668
2,278145.708917,255091.529800
3,70.778414,59.290460
4,0.093325,0.089803
5,0.074890,0.077161
6,293090.637508,281854.067556
7,78.258602,76.270895
8,239905.587200,225569.258315
9,67.904876,65.870865


In [ ]:
# cif lstm
def build_lstm_model(input_len, output_len):
  inputs = Input(shape=(input_len,1))
  x = LSTM(units = 8, return_sequences = False)(inputs)
  x = Dropout(0.5)(x)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

run_global_domain_global(cif_data, cif_info, input_len = 12, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 2,
                             random_seed = 3407)

num_series: 72
total number of windows: 5470
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 12, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 8)                   │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

 Trainable params: 361 (1.41 KB)

...

 Total params: 445 (1.74 KB)

 Trainable params: 89 (356.00 B)

 Non-trainable params: 356 (1.39 KB)

None
Epoch 1/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.7253
Epoch 2/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.2379
Epoch 3/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.9213
Epoch 4/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.7042
Epoch 5/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5874
Epoch 6/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5126
Epoch 7/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4437
Epoch 8/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4194
Epoch 9/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4048
Epoch 10/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3738
Epoch 11/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.3768
Epoch 12/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3144
Epoch 13/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.3243
Epoch 14/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3040
Epoch 15/40
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

,res_without_adj,res_with_adj
0,338097.605864,311803.506198
1,95.016699,69.929753
2,261428.802899,238722.980331
3,78.694409,58.024549
4,0.100958,0.092104
5,0.081004,0.069003
6,268568.973354,248097.483985
7,87.127801,75.732632
8,215052.708069,199321.156630
9,72.228685,62.791573


In [ ]:
hospital_data = pd.read_csv('hospital_data.csv')
hospital_info = pd.read_csv('hospital_info.csv')

In [ ]:
# hospital mlp
def build_mlp_model(input_len, output_len):
  inputs = Input(shape=(input_len,))
  x = Dense(16, activation="tanh")(inputs)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

run_global_domain_global(hospital_data, hospital_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 42)

num_series: 767
total number of windows: 36816
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

...

 Total params: 861 (3.36 KB)

 Trainable params: 461 (1.80 KB)

 Non-trainable params: 400 (1.56 KB)

None
Epoch 1/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.9503
Epoch 2/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.3174
Epoch 3/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2027
Epoch 4/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1604
Epoch 5/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1364
Epoch 6/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1191
Epoch 7/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1028
Epoch 8/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0841
Epoch 9/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0639
Epoch 10/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0467
Epoch 11/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0349
Epoch 12/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0261
Epoch 13/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0185
Epoch 14/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.0114
Epoch 15/40
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

,res_without_adj,res_with_adj
0,20.709554,21.111381
1,7.838620,7.868630
2,16.995502,17.282954
3,6.347411,6.384546
4,0.163937,0.165089
5,0.155279,0.154854
6,20.673625,20.926499
7,7.832536,7.944430
8,17.650893,17.815643
9,6.600078,6.664795


In [ ]:
# hospital lstm
def build_lstm_model(input_len, output_len):
  inputs = Input(shape=(input_len,1))
  x = LSTM(units = 8, return_sequences = False)(inputs)
  x = Dropout(0.5)(x)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

run_global_domain_global(hospital_data, hospital_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series: 767
total number of windows: 36816
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 8)                   │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

 Trainable params: 361 (1.41 KB)

...

 Total params: 529 (2.07 KB)

 Trainable params: 173 (692.00 B)

 Non-trainable params: 356 (1.39 KB)

None
Epoch 1/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.9450
Epoch 2/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3351
Epoch 3/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.1868
Epoch 4/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.1362
Epoch 5/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.1154
Epoch 6/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0837
Epoch 7/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0652
Epoch 8/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.0401
Epoch 9/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0466
Epoch 10/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.0373
Epoch 11/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.0248
Epoch 12/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.0113
Epoch 13/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.0005
Epoch 14/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.9972
Epoch 15/40
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

,res_without_adj,res_with_adj
0,22.325054,21.971037
1,7.769994,7.769994
2,18.319623,17.969398
3,6.346960,6.366622
4,0.164555,0.165042
5,0.155277,0.155277
6,22.055531,21.961244
7,7.724806,7.752780
8,18.739189,18.635183
9,6.557925,6.611791


In [ ]:
def run_cluster_models_m3(required_num_of_clusters, hetero_series_idx, cluster_labels,
                       xtrain_lst, xtest_lst, num_series,
                       input_len, output_len,
                       model, epochs_stage_2, batch_size_stage_2,
                       pred_lst, train_true_lst,
                       test_pred_lst, test_true_lst,
                       num_window_per_series_train, num_window_per_series_test,
                       random_seed):
  count_cluster, num_windosw_cluster, num_hetero_cluster = [], [], []
  test_pred_with_adj_lst = copy.deepcopy(test_pred_lst)

  for idx_cluster in range(required_num_of_clusters):
    cluster_i = hetero_series_idx[np.where(cluster_labels == idx_cluster)]
    count_cluster.append(len(cluster_i))

    cluster_lst = []
    for i in cluster_i:
      cluster_lst.append(xtrain_lst[i])
    cluster_test_lst = []
    for i in cluster_i:
      cluster_test_lst.append(xtest_lst[i])

    # prepare training data
    train_data_frame_clu, norm_train_data_frame_clu, num_window_per_series_train_clu, mean_vec_train_clu, std_vec_train_clu = \
    split_windows(cluster_lst, len(cluster_i), input_len, output_len)
    num_windosw_cluster.append(np.sum(num_window_per_series_train_clu))
    # [batch, timesteps, feature]
    train_X_clu = norm_train_data_frame_clu[:,:-output_len].reshape((-1,input_len,1))
    train_Y_clu = norm_train_data_frame_clu[:,-output_len].reshape((-1,output_len,1))

    # if np.sum(num_window_per_series_train_clu) <= 4000:
    #   model_clu = build_cluster_model(model, output_len, complex=False)
    # else:
    #   model_clu = build_cluster_model(model, output_len, complex=True)
    model_clu = build_cluster_model_V1(model, input_len, output_len)
    print(model_clu.summary())
    # Compile the model
    model_clu.compile(loss = 'mean_squared_error', optimizer = 'adam')
    # Fine-tune the model
    set_global_determinism(seed=random_seed)
    model_clu.fit(train_X_clu, train_Y_clu, epochs=epochs_stage_2, batch_size=batch_size_stage_2)

    ##### evaluation on training dataset for this cluster
    train_pred_clu, train_true_clu = pred_and_results(train_X_clu, model_clu, train_data_frame_clu, output_len,
                                                      mean_vec_train_clu, std_vec_train_clu)
    train_pred_lst_clu = [pred_lst[i] for i in cluster_i]
    train_true_lst_clu = [train_true_lst[i] for i in cluster_i]
    # train, without adj
    train_metrics_clu_without_adj = metrics(len(cluster_i), train_true_lst_clu, train_pred_lst_clu)

    train_pred_clu_lst = split_to_series(train_pred_clu, num_window_per_series_train[cluster_i], len(cluster_i))
    # train, with adj
    train_metrics_clu_with_adj = metrics(len(cluster_i), train_true_lst_clu, train_pred_clu_lst)

    ##### detect heteroscedastic series for this cluster
    diff_lst_with_adj = [np.array([train_true_lst_clu[i] - train_pred_clu_lst[i]]).reshape(-1) for i in range(len(train_true_lst_clu))]
    # Initialize p-value vector
    p_vec_clu = np.zeros(len(cluster_i))
    # Perform the Ljung-Box test for each series
    for i in range(len(cluster_i)):
        lb_test = acorr_ljungbox(diff_lst_with_adj[i], lags=int(np.log(len(diff_lst_with_adj[i]))), return_df=True)
        p_vec_clu[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
    # Count and get indices of series with p-values < 0.05
    hetero_series_idx_clu = np.where(p_vec_clu < 0.05)[0]
    print(f"Number of heteroscedastic series: {len(hetero_series_idx_clu)}")
    num_hetero_cluster.append(len(hetero_series_idx_clu))

    ##### evaluation on test dataset for this cluster
    test_pred_lst_clu = [test_pred_lst[i] for i in cluster_i]
    test_true_lst_clu = [test_true_lst[i] for i in cluster_i]
    # test, without adj
    test_metrics_clu_without_adj = metrics(len(cluster_i), test_true_lst_clu, test_pred_lst_clu)

    # test, with adj
    test_data_frame_clu, norm_test_data_frame_clu, num_window_per_series_test_clu, mean_vec_test_clu, std_vec_test_clu = \
    split_windows(cluster_test_lst, len(cluster_i), input_len, output_len)
    test_X_clu = norm_test_data_frame_clu[:,:-output_len].reshape((-1,input_len,1))
    test_pred_clu, _ = pred_and_results(test_X_clu, model_clu, test_data_frame_clu, output_len,
                                        mean_vec_test_clu, std_vec_test_clu)
    test_pred_clu_lst = split_to_series(test_pred_clu, num_window_per_series_test[cluster_i], len(cluster_i))
    test_metrics_clu_with_adj = metrics(len(cluster_i), test_true_lst_clu, test_pred_clu_lst)

    ##### substitute and update the predictions of this cluster
    for i in cluster_i:
      test_pred_with_adj_lst[i] = test_pred_clu_lst[np.where(cluster_i == i)[0][0]]

    df = pd.DataFrame({'train_metrics_clu_without_adj': train_metrics_clu_without_adj,
                      'train_metrics_clu_with_adj': train_metrics_clu_with_adj,
                      'test_metrics_clu_without_adj': test_metrics_clu_without_adj,
                      'test_metrics_clu_with_adj': test_metrics_clu_with_adj})
    print('cluster', idx_cluster, df)

  # test_metrics_with_adj = metrics(num_series, test_true_lst, test_pred_with_adj_lst)
  print(count_cluster, num_windosw_cluster, num_hetero_cluster)
  return([num_series, test_true_lst, test_pred_with_adj_lst])

In [ ]:
def stage_2_m3(xtrain_lst, xtest_lst, num_series, input_len, output_len,
            model, epochs_stage_2, batch_size_stage_2,
            pred_lst, train_true_lst, test_pred_lst, test_true_lst,
            hetero_series_idx, diff_lst,
            seasonal_period, random_seed,
            num_window_per_series_train, num_window_per_series_test,
            required_num_of_clusters = 5):
  set_global_determinism(seed=random_seed)
  print('clustering into', required_num_of_clusters, 'clusters')
  hetero_df = pd.DataFrame({'unique_id': [], 'ds': [], 'y': []})
  for i in hetero_series_idx:
    hetero_df_i = {'unique_id': [int(i) for j in range(len(diff_lst[i]))],
                  'ds': list(xtrain_lst[i].loc[(input_len):, 'time']),
                  'y': list(diff_lst[i])}
    hetero_df = pd.concat([hetero_df, pd.DataFrame(hetero_df_i)], axis=0, ignore_index=True)
  selected_features = tsfeatures(hetero_df, freq=seasonal_period,
                                 features=[acf_features, pacf_features, entropy, lumpiness, stl_features, arch_stat,
                                           nonlinearity, unitroot_kpss, unitroot_pp, holt_parameters, hw_parameters])
  # Select specific features
  selected_features = selected_features[[
      'hw_alpha', 'hw_beta', 'hw_gamma', 'alpha', 'beta',
      'trend', 'spike', 'linearity', 'curvature', 'entropy',
      'x_acf1', 'lumpiness'
  ]]
  # Apply Min-Max normalization
  scaler = MinMaxScaler()
  normalized_features = scaler.fit_transform(selected_features)
  normalized_features = pd.DataFrame(normalized_features, columns=selected_features.columns)
  # Remove columns containing NaN values
  normalized_features = normalized_features.dropna(axis=1)
  # divide domains
  kmeans = KMeans(n_clusters=required_num_of_clusters, random_state=0).fit(normalized_features)
  cluster_labels = kmeans.labels_

  print('building models for each cluster')
  num_series, test_true_lst, test_pred_with_adj_lst = run_cluster_models_m3(required_num_of_clusters, hetero_series_idx, cluster_labels,
                                             xtrain_lst, xtest_lst, num_series, input_len, output_len,
                                             model, epochs_stage_2, batch_size_stage_2,
                                             pred_lst, train_true_lst,
                                             test_pred_lst, test_true_lst,
                                             num_window_per_series_train, num_window_per_series_test,
                                             random_seed)
  return([num_series, test_true_lst, test_pred_with_adj_lst])

In [ ]:
def run_global_domain_global_m3(data, info, input_len, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 16,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 5,
                             random_seed = 42):
  num_series = info.shape[0]
  print('num_series:', num_series)

  data_lst = []
  for i in range(num_series):
    data_lst.append(data[data['idx']==(i+1)])

  xtest_idx = np.array(info['test_idx'])
  xtest_lst, xtrain_lst = [], []
  for i in range(num_series):
    xtrain_lst.append(data_lst[i][data_lst[i]['time']<xtest_idx[i]].reset_index(drop=True))
    xtest_lst.append(data_lst[i][data_lst[i]['time']>=xtest_idx[i] - input_len].reset_index(drop=True))

  train_data_frame, norm_train_data_frame, num_window_per_series_train, mean_vec_train, std_vec_train = \
  split_windows(xtrain_lst, num_series, input_len, output_len)
  # Calculate the total number of windows
  num_window_total_train = np.sum(num_window_per_series_train)
  print('total number of windows:', num_window_total_train)

  # [batch, timesteps, feature]
  train_X = norm_train_data_frame[:,:-output_len].reshape((-1,input_len,1))
  train_Y = norm_train_data_frame[:,-output_len].reshape((-1,output_len,1))

  tf.keras.backend.clear_session()
  # Call the above function with seed value
  set_global_determinism(seed=random_seed)

  if load_model_name == None:
    print('start training')
    model = build_model(input_len, output_len)
    history = model.fit(train_X, train_Y, batch_size = batch_size_stage_1, epochs = epochs_stage_1,
                        verbose=1, shuffle=False)
  else:
    print('loading model')
    model = tf.keras.models.load_model(load_model_name)
    print('done')

  orig_pred_train, train_true = \
  pred_and_results(train_X, model, train_data_frame, output_len, mean_vec_train, std_vec_train)
  train_rmse_without_adj = np.sqrt(np.mean((orig_pred_train - train_true)**2))
  print('train_rmse_without_adj:', train_rmse_without_adj)

  # detect heteroscedastic series
  diff = train_true / orig_pred_train # multiplicative model
  diff_lst = split_to_series(diff, num_window_per_series_train, num_series)
  pred_lst = split_to_series(orig_pred_train, num_window_per_series_train, num_series)
  # Initialize p-value vector
  p_vec = np.zeros(num_series)
  # Perform the Ljung-Box test for each series
  for i in range(num_series):
      lb_test = acorr_ljungbox(diff_lst[i], lags=int(np.log(len(diff_lst[i]))), return_df=True)
      p_vec[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
  # Count and get indices of series with p-values < 0.05
  hetero_series_idx = np.where(p_vec < 0.05)[0]
  print(f"Number of heteroscedastic series: {len(hetero_series_idx)}")

  train_true_lst = split_to_series(train_data_frame[:, -output_len], num_window_per_series_train, num_series)
  # test for the entire dataset
  print('start testing')
  test_data_frame, norm_test_data_frame, num_window_per_series_test, mean_vec_test, std_vec_test = \
  split_windows(xtest_lst, num_series, input_len, output_len)
  test_X = norm_test_data_frame[:, :input_len].reshape((-1,input_len,1))
  test_pred, test_true = \
  pred_and_results(test_X, model, test_data_frame, output_len, mean_vec_test, std_vec_test)
  test_pred_lst = split_to_series(test_pred, num_window_per_series_test, num_series)
  test_true_lst = split_to_series(test_true, num_window_per_series_test, num_series)
  test_metrics_without_adj = metrics(num_series, test_true_lst, test_pred_lst)

  # adj model
  print('entering stage 2')
  num_series, test_true_lst, test_pred_with_adj_lst = stage_2_m3(xtrain_lst, xtest_lst, num_series, input_len, output_len,
                                  model, epochs_stage_2, batch_size_stage_2,
                                  pred_lst, train_true_lst, test_pred_lst, test_true_lst,
                                  hetero_series_idx, diff_lst,
                                  seasonal_period, random_seed,
                                  num_window_per_series_train, num_window_per_series_test,
                                  required_num_of_clusters = required_num_of_clusters)
  # metrics_res = {'res_without_adj': test_metrics_without_adj,
  #              'res_with_adj': test_metrics_with_adj}
  # metrics_res_df = pd.DataFrame(metrics_res)
  return([num_series, test_true_lst, test_pred_lst, test_pred_with_adj_lst])

In [ ]:
m3_1_data = pd.read_csv('m3_1_data.csv')
m3_1_info = pd.read_csv('m3_1_info.csv')
m3_2_data = pd.read_csv('m3_2_data.csv')
m3_2_info = pd.read_csv('m3_2_info.csv')
m3_3_data = pd.read_csv('m3_3_data.csv')
m3_3_info = pd.read_csv('m3_3_info.csv')
m3_4_data = pd.read_csv('m3_4_data.csv')
m3_4_info = pd.read_csv('m3_4_info.csv')
m3_5_data = pd.read_csv('m3_5_data.csv')
m3_5_info = pd.read_csv('m3_5_info.csv')

In [ ]:
def build_mlp_model(input_len, output_len):
  inputs = Input(shape=(input_len,))
  x = Dense(16, activation="tanh")(inputs)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
num_series_m3_1, true_demand_lst_m3_1, pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1 = \
run_global_domain_global_m3(m3_1_data, m3_1_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series: 111
total number of windows: 9028
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 16)             │            400 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_1 (Dense)       │ (None, 24)             │            408 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 24)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 24)             │              0 │ new_dense_1[0][0],     │
│                           │                        │                │ reshape_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_2 (Dense)       │ (None, 2)              │             50 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_3 (Dense)       │ (None, 1)              │              3 │ new_dense_2[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 861 (3.36 KB)

 Trainable params: 461 (1.80 KB)

 Non-trainable params: 400 (1.56 KB)

None
Epoch 1/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.3531
Epoch 2/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8387
Epoch 3/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4975
Epoch 4/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3826
Epoch 5/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3281
Epoch 6/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2958
Epoch 7/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2743
Epoch 8/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2581
Epoch 9/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2449
Epoch 10/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2337
Epoch 11/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2241
Epoch 12/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2158
Epoch 13/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2084
Epoch 14/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2019
Epoch 15/40
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [ ]:
def metrics_m3(num_series, true_demand_lst, pred_test_demand_lst_without_adj, pred_test_demand_lst_with_adj):
  res_without_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst_without_adj)
  res_with_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst_with_adj)
  metrics_res = {'res_without_adj': res_without_adj,
               'res_with_adj': res_with_adj}
  df = pd.DataFrame(metrics_res)
  return(df)

In [ ]:
metrics_m3(num_series_m3_1, true_demand_lst_m3_1,
           pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1)

,res_without_adj,res_with_adj
0,243.297371,245.581771
1,47.580275,44.266090
2,187.938693,189.469131
3,36.581207,36.581207
4,0.040851,0.041101
5,0.005637,0.005637
6,252.347411,251.600776
7,39.576784,39.062205
8,202.004401,200.077680
9,33.364113,33.364113


In [ ]:
num_series_m3_2, true_demand_lst_m3_2, pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2 = \
run_global_domain_global_m3(m3_2_data, m3_2_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series_m3_4, true_demand_lst_m3_4, pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4 = \
run_global_domain_global_m3(m3_4_data, m3_4_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series: 145
total number of windows: 11948
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 16)             │            400 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_1 (Dense)       │ (None, 24)             │            408 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 24)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 24)             │              0 │ new_dense_1[0][0],     │
│                           │                        │                │ reshape_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_2 (Dense)       │ (None, 2)              │             50 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_3 (Dense)       │ (None, 1)              │              3 │ new_dense_2[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 861 (3.36 KB)

 Trainable params: 461 (1.80 KB)

 Non-trainable params: 400 (1.56 KB)

None
Epoch 1/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.4782
Epoch 2/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8191
Epoch 3/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7321
Epoch 4/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6725
Epoch 5/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6376
Epoch 6/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.6173
Epoch 7/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6033
Epoch 8/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5921
Epoch 9/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.5827
Epoch 10/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5747
Epoch 11/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5678
Epoch 12/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5619
Epoch 13/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5566
Epoch 14/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.5520
Epoch 15/40
925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

In [ ]:
metrics_m3(num_series_m3_2, true_demand_lst_m3_2,
           pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2)

,res_without_adj,res_with_adj
0,558.516365,553.708625
1,198.750767,198.750767
2,441.299048,434.518001
3,159.919177,163.555705
4,0.070901,0.070028
5,0.021754,0.024036
6,574.034492,564.049218
7,197.641609,199.992605
8,473.542851,462.328447
9,172.672273,168.991335


In [ ]:
num_series_m3_3, true_demand_lst_m3_3, pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3 = \
run_global_domain_global_m3(m3_3_data, m3_3_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 20,
                             required_num_of_clusters = 5,
                             random_seed = 3407)
metrics_m3(num_series_m3_3, true_demand_lst_m3_3,
           pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3)

num_series: 334
total number of windows: 32739
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

...

 Total params: 861 (3.36 KB)

 Trainable params: 461 (1.80 KB)

 Non-trainable params: 400 (1.56 KB)

None
Epoch 1/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.3712
Epoch 2/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.0277
Epoch 3/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.9897
Epoch 4/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9660
Epoch 5/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.9490
Epoch 6/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9367
Epoch 7/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9275
Epoch 8/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.9203
Epoch 9/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9145
Epoch 10/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9096
Epoch 11/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.9051
Epoch 12/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9010
Epoch 13/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.8972
Epoch 14/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8935
Epoch 15/20
947/947 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

,res_without_adj,res_with_adj
0,540.603638,543.366216
1,334.970485,340.647351
2,421.787818,423.296890
3,263.022020,268.734323
4,0.083506,0.083630
5,0.054440,0.056286
6,500.538379,503.800483
7,322.454076,323.217188
8,408.311260,410.182076
9,265.227770,267.917597


In [ ]:
metrics_m3(num_series_m3_4, true_demand_lst_m3_4,
           pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4)

,res_without_adj,res_with_adj
0,259.959559,258.794944
1,119.548057,119.417084
2,205.595277,203.867105
3,95.908987,92.314167
4,0.032908,0.032693
5,0.015699,0.015636
6,258.350555,256.066711
7,120.086866,112.414743
8,212.815446,209.950512
9,98.421847,91.109693


In [ ]:
num_series_m3_5, true_demand_lst_m3_5, pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5 = \
run_global_domain_global_m3(m3_5_data, m3_5_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_mlp_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 20,
                             required_num_of_clusters = 3,
                             random_seed = 3407)
metrics_m3(num_series_m3_5, true_demand_lst_m3_5,
           pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5)

num_series: 474
total number of windows: 24009
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

...

 Total params: 861 (3.36 KB)

 Trainable params: 461 (1.80 KB)

 Non-trainable params: 400 (1.56 KB)

None
Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1947
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1000 
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0200 
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9521 
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8971 
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8538  
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8195 
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7914 
Epoch 9/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7673 
Epoch 10/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7460 
Epoch 11/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7265 
Epoch 12/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7084  
Epoch 13/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6912 
Epoch 14/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6745  
Epoch 15/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.658

,res_without_adj,res_with_adj
0,869.409429,869.634419
1,720.537390,714.089453
2,695.455573,696.058107
3,583.559266,583.559266
4,0.205506,0.205607
5,0.180506,0.181223
6,874.541173,874.309698
7,750.999763,746.021730
8,727.456367,727.567240
9,618.828031,614.350084


In [ ]:
num_series_m3 = num_series_m3_1 + num_series_m3_2 + num_series_m3_3 + num_series_m3_4 + num_series_m3_5
true_demand_lst_m3 = true_demand_lst_m3_1 + true_demand_lst_m3_2 + true_demand_lst_m3_3 + true_demand_lst_m3_4 + true_demand_lst_m3_5
pred_test_demand_lst_without_adj_m3 = pred_test_demand_lst_without_adj_m3_1 + pred_test_demand_lst_without_adj_m3_2 + pred_test_demand_lst_without_adj_m3_3 + pred_test_demand_lst_without_adj_m3_4 + pred_test_demand_lst_without_adj_m3_5
pred_test_demand_lst_with_adj_m3 = pred_test_demand_lst_with_adj_m3_1 + pred_test_demand_lst_with_adj_m3_2 + pred_test_demand_lst_with_adj_m3_3 + pred_test_demand_lst_with_adj_m3_4 + pred_test_demand_lst_with_adj_m3_5

In [ ]:
metrics_m3(num_series_m3, true_demand_lst_m3,
           pred_test_demand_lst_without_adj_m3, pred_test_demand_lst_with_adj_m3)

,res_without_adj,res_with_adj
0,568.139497,568.301149
1,336.822980,340.568841
2,450.231364,449.822256
3,266.801018,269.214155
4,0.109290,0.109235
5,0.054831,0.056037
6,562.182610,561.264386
7,328.167800,329.175721
8,464.153270,462.658788
9,270.498211,271.837659


In [ ]:
# m3 lstm
def build_lstm_model(input_len, output_len):
  inputs = Input(shape=(input_len,1))
  x = LSTM(units = 16, return_sequences = False)(inputs)
  x = Dropout(0.5)(x)
  x = Dense(4)(x)
  outputs = Dense(output_len)(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
num_series_m3_1, true_demand_lst_m3_1, pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1 = \
run_global_domain_global_m3(m3_1_data, m3_1_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series: 111
total number of windows: 9028
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,225 (4.79 KB)

 Trainable params: 1,225 (4.79 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 16)             │          1,152 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 16)             │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 4)              │             68 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_1 (Dense)       │ (None, 24)             │            120 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 24)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 24)             │              0 │ new_dense_1[0][0],     │
│                           │                        │                │ reshape_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_2 (Dense)       │ (None, 2)              │             50 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_3 (Dense)       │ (None, 1)              │              3 │ new_dense_2[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,393 (5.44 KB)

 Trainable params: 173 (692.00 B)

 Non-trainable params: 1,220 (4.77 KB)

None
Epoch 1/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2.6025
Epoch 2/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.0622
Epoch 3/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.7054
Epoch 4/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.5681
Epoch 5/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.4969
Epoch 6/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4458
Epoch 7/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4052
Epoch 8/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3608
Epoch 9/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3207
Epoch 10/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2860
Epoch 11/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2560
Epoch 12/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.2382
Epoch 13/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2240
Epoch 14/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2176
Epoch 15/40
470/470 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

In [ ]:
metrics_m3(num_series_m3_1, true_demand_lst_m3_1,
           pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1)

,res_without_adj,res_with_adj
0,304.954442,297.824173
1,54.117659,44.801728
2,221.800143,212.988590
3,46.488944,35.874640
4,0.040942,0.040568
5,0.006764,0.006032
6,306.751005,297.241958
7,49.435802,43.151018
8,230.997013,220.559255
9,40.814085,36.849695


In [ ]:
num_series_m3_2, true_demand_lst_m3_2, pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2 = \
run_global_domain_global_m3(m3_2_data, m3_2_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series_m3_3, true_demand_lst_m3_3, pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3 = \
run_global_domain_global_m3(m3_3_data, m3_3_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series_m3_4, true_demand_lst_m3_4, pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4 = \
run_global_domain_global_m3(m3_4_data, m3_4_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series_m3_5, true_demand_lst_m3_5, pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5 = \
run_global_domain_global_m3(m3_5_data, m3_5_info, input_len = 24, output_len = 1, seasonal_period = 12,
                             load_model_name = None, build_model = build_lstm_model,
                             batch_size_stage_1 = 32, batch_size_stage_2 = 4,
                             epochs_stage_1 = 30, epochs_stage_2 = 40,
                             required_num_of_clusters = 3,
                             random_seed = 3407)

num_series: 145
total number of windows: 11948
start training


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,225 (4.79 KB)

 Trainable params: 1,225 (4.79 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 16)             │          1,152 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 16)             │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 4)              │             68 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_1 (Dense)       │ (None, 24)             │            120 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_2 (Reshape)       │ (None, 24)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 24)             │              0 │ new_dense_1[0][0],     │
│                           │                        │                │ reshape_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_2 (Dense)       │ (None, 2)              │             50 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_dense_3 (Dense)       │ (None, 1)              │              3 │ new_dense_2[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,393 (5.44 KB)

 Trainable params: 173 (692.00 B)

 Non-trainable params: 1,220 (4.77 KB)

None
Epoch 1/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 2.2008
Epoch 2/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.0142
Epoch 3/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.8634
Epoch 4/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.7794
Epoch 5/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.6982
Epoch 6/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6257
Epoch 7/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.5903
Epoch 8/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.5482
Epoch 9/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.5070
Epoch 10/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4905
Epoch 11/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4597
Epoch 12/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.4725
Epoch 13/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.4339
Epoch 14/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.4151
Epoch 15/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.4088
Epoch 16/40
91

In [ ]:
metrics_m3(num_series_m3_2, true_demand_lst_m3_2,
           pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2)

,res_without_adj,res_with_adj
0,579.591234,541.701496
1,206.887176,199.319238
2,473.992841,435.037816
3,178.243915,159.325229
4,0.075258,0.070288
5,0.025182,0.023090
6,568.443702,533.119986
7,206.106928,193.355966
8,476.317009,442.024217
9,176.287665,156.298254


In [ ]:
metrics_m3(num_series_m3_3, true_demand_lst_m3_3,
           pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3)

,res_without_adj,res_with_adj
0,605.464445,583.322296
1,388.153681,357.258863
2,470.676016,454.004822
3,302.816136,282.529399
4,0.089433,0.086110
5,0.063936,0.057968
6,567.332309,549.150569
7,355.191236,337.735452
8,464.430953,450.107145
9,297.080580,277.902210


In [ ]:
metrics_m3(num_series_m3_4, true_demand_lst_m3_4,
           pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4)

,res_without_adj,res_with_adj
0,312.372697,282.598713
1,125.827101,112.967515
2,246.575822,223.091246
3,96.941809,91.749668
4,0.038698,0.035457
5,0.015614,0.014924
6,313.760603,280.913075
7,115.696592,106.570643
8,257.222556,229.964844
9,91.637709,88.399763


In [ ]:
metrics_m3(num_series_m3_5, true_demand_lst_m3_5,
           pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5)

,res_without_adj,res_with_adj
0,831.804499,834.624187
1,706.558233,712.547050
2,667.168626,668.864671
3,568.787430,576.347540
4,0.198213,0.198238
5,0.173445,0.174532
6,831.651865,835.764165
7,707.779459,720.215798
8,693.136461,696.265229
9,592.005680,602.266752


In [ ]:
num_series_m3 = num_series_m3_1 + num_series_m3_2 + num_series_m3_3 + num_series_m3_4 + num_series_m3_5
true_demand_lst_m3 = true_demand_lst_m3_1 + true_demand_lst_m3_2 + true_demand_lst_m3_3 + true_demand_lst_m3_4 + true_demand_lst_m3_5
pred_test_demand_lst_without_adj_m3 = pred_test_demand_lst_without_adj_m3_1 + pred_test_demand_lst_without_adj_m3_2 + pred_test_demand_lst_without_adj_m3_3 + pred_test_demand_lst_without_adj_m3_4 + pred_test_demand_lst_without_adj_m3_5
pred_test_demand_lst_with_adj_m3 = pred_test_demand_lst_with_adj_m3_1 + pred_test_demand_lst_with_adj_m3_2 + pred_test_demand_lst_with_adj_m3_3 + pred_test_demand_lst_with_adj_m3_4 + pred_test_demand_lst_with_adj_m3_5

In [ ]:
metrics_m3(num_series_m3, true_demand_lst_m3,
           pred_test_demand_lst_without_adj_m3, pred_test_demand_lst_with_adj_m3)

,res_without_adj,res_with_adj
0,590.008292,574.285981
1,372.033395,353.927182
2,467.822785,454.219586
3,293.702233,277.074227
4,0.109996,0.107909
5,0.063486,0.058846
6,579.984725,565.050610
7,358.744301,338.710635
8,478.653120,465.617823
9,297.640836,281.159506
